### Obtencion de datos

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

# Especifica la ruta al archivo Excel
archivo_excel = 'extract_glosa.xlsx'

# Lee el archivo Excel en un DataFrame de Pandas
df = pd.read_excel(archivo_excel)

### Obtener Dataframe con filas por separado por cada glosa

In [33]:
ns = {'ns': 'http://www.sii.cl/SiiDte'}

filas_sin = []
filas_con = []

# Funcion para obtener un elemento
def get_element_value(element, tag):
    return element.find(f'ns:{tag}', ns).text if element.find(f'ns:{tag}', ns) is not None else None


# Recorrer el dataframe
for xml in df['convert_from'].head(4):

    root = ET.fromstring(xml)
    namespace = {'default': 'http://www.sii.cl/SiiDte'}

    # Obtener los valores "globales" del xml
    Folio = root.find('.//{http://www.sii.cl/SiiDte}Folio').text
    RutEmisor = root.find('.//{http://www.sii.cl/SiiDte}RutEmisor').text

    # Obtener todos los detalles del xml <detalle>
    detalles = root.findall('.//ns:Detalle', ns)

    # Recorrer cada detalle de los detalles
    for detalle in detalles:
        NroLinDet = get_element_value(detalle, 'NroLinDet')
        NmbItem = get_element_value(detalle, 'NmbItem')
        MontoItem = get_element_value(detalle, 'MontoItem')
        DscItem = get_element_value(detalle, 'DscItem')
        # Crear una fila para agregar al dataframe
        # Si esta el elemento <DscItem> se agrega al df que contiene las intancias con DscItem
        if DscItem is not None:
            nuevaFila = {
            "RutEmisor": RutEmisor,
            "Folio": Folio,
            "N_Item":NroLinDet,
            "NmbItem": NmbItem,
            "DscItem": DscItem,
            "MontoItem": MontoItem,
            }
            filas_con.append(nuevaFila)
        # Si no esta el elemento <DscItem> se agrega la fila al df que no tiene 
        else:
            nuevaFila = {
            "RutEmisor": RutEmisor,
            "Folio": Folio,
            "N_Item":NroLinDet,
            "NmbItem": NmbItem,
            "DscItem": None,
            "MontoItem": MontoItem,
            }
            filas_sin.append(nuevaFila)

# agregar las nuevas filas a los distintos df
df_con = pd.DataFrame(filas_con) # df con DscItem
df_sin = pd.DataFrame(filas_sin) # df sin DscItem

In [26]:
df_con.head()

,RutEmisor,Folio,N_Item,NmbItem,DscItem,MontoItem
0,77566800-8,440,1,Arriendo local 2 Puerto,montt valor proporcional mes de Enero uf 140,5156309
1,76789922-K,937,1,Usage: Remote Hands 1 Day,Ticket: 822574: 12/19/23: Requested by: carlos...,916160


In [27]:
df_sin.head()

,RutEmisor,Folio,N_Item,NmbItem,DscItem,MontoItem
0,16383874-5,89,1,Arriendo Álvaro Casanova,None,898498
1,96689310-9,50872845,1,COMISION POR VENTAS NACIONALES. EMISOR R.U.T.:...,None,114
2,96689310-9,50872845,2,SERVICIO TBK 12/2023,None,10807
3,96689310-9,50872845,3,COMISIONES POR VENTAS NACIONALES,None,2922


### Juntar NmbItem y DscItem para convertirlo en la glosa

#### Seccion para el df que `no tiene DscItem`

In [34]:
# Elimino la columna DscItem porque no tiene nada en ese campo
df_sin = df_sin.drop(columns=['DscItem'])
# Renombro las columna NmbItem que es la unica glosa que hay
df_sin = df_sin.rename(columns={'NmbItem': 'Glosa'})

#### Seccion para el df que `tiene DscItem`

In [37]:
def combinar_columnas(df, col1, col2, nueva_columna):
    df[nueva_columna] = df[col1].astype(str) + " " + df[col2].astype(str)
    df = df.drop(columns=[col1, col2])
    return df

def cambiar_lugar_columnas(df, col1, col2):
    if col1 in df.columns and col2 in df.columns:
        cols = list(df.columns)
        col1_index, col2_index = cols.index(col1), cols.index(col2)
        
        # Intercambiar los nombres de las columnas en la lista
        cols[col1_index], cols[col2_index] = cols[col2_index], cols[col1_index]
        
        # Reordenar el DataFrame según la nueva lista de columnas
        df = df[cols]
        print(f"Columnas '{col1}' y '{col2}' han sido intercambiadas.")
    else:
        print(f"Una o ambas columnas '{col1}' y '{col2}' no existen en el DataFrame.")
    
    return df

df_con = combinar_columnas(df_con,'NmbItem','DscItem','Glosa')
df_con = cambiar_lugar_columnas(df_con, 'MontoItem', 'Glosa')

Columnas 'MontoItem' y 'Glosa' han sido intercambiadas.


In [38]:
df_con.head()

,RutEmisor,Folio,N_Item,Glosa,MontoItem
0,77566800-8,440,1,Arriendo local 2 Puerto montt valor proporc...,5156309
1,76789922-K,937,1,Usage: Remote Hands 1 Day Ticket: 822574: 12/1...,916160


In [39]:
df_sin.head()

,RutEmisor,Folio,N_Item,Glosa,MontoItem
0,16383874-5,89,1,Arriendo Álvaro Casanova,898498
1,96689310-9,50872845,1,COMISION POR VENTAS NACIONALES. EMISOR R.U.T.:...,114
2,96689310-9,50872845,2,SERVICIO TBK 12/2023,10807
3,96689310-9,50872845,3,COMISIONES POR VENTAS NACIONALES,2922


In [40]:
df_con.head()

,RutEmisor,Folio,N_Item,Glosa,MontoItem
0,77566800-8,440,1,Arriendo local 2 Puerto montt valor proporc...,5156309
1,76789922-K,937,1,Usage: Remote Hands 1 Day Ticket: 822574: 12/1...,916160


### Juntar los Dataframe

In [41]:
df_new = pd.concat([df_con, df_sin], axis=0, ignore_index=False)

In [43]:
df_new.head(10)

,RutEmisor,Folio,N_Item,Glosa,MontoItem
0,77566800-8,440,1,Arriendo local 2 Puerto montt valor proporc...,5156309
1,76789922-K,937,1,Usage: Remote Hands 1 Day Ticket: 822574: 12/1...,916160
0,16383874-5,89,1,Arriendo Álvaro Casanova,898498
1,96689310-9,50872845,1,COMISION POR VENTAS NACIONALES. EMISOR R.U.T.:...,114
2,96689310-9,50872845,2,SERVICIO TBK 12/2023,10807
3,96689310-9,50872845,3,COMISIONES POR VENTAS NACIONALES,2922
